### This NoteBook provides some Notes about Fully Connected Neural Network

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset

In [2]:
## config
device  = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
### HyperParameters
learning_rate = 0.001
n_epochs = 10
n_batch_size = 32

input_size = 784
hidden_size = 100
num_classes = 10

#### Load the MNIST DataSet and create the DataLoader

In [4]:
## make some tranformation
data_transform = transforms.Compose([transforms.ToTensor()])                              


## load the dataset
train_dataset = torchvision.datasets.MNIST(root='data', download=True, 
                                           train=True, transform=data_transform)
test_dataset = torchvision.datasets.MNIST(root='data', download=True, 
                                           train=False, transform=data_transform)


## create the dataloader
train_loader = DataLoader(dataset=train_dataset, batch_size=n_batch_size, 
                          shuffle=True, num_workers=0)
test_loader = DataLoader(dataset=test_dataset, batch_size=n_batch_size, 
                          shuffle=False, num_workers=0)

In [5]:
## check our work
examples = iter(train_loader)
features, label = examples.next()
print('features.shape -->', features.shape)  ## note the shape
print('label.shape -->', label.shape)

features.shape --> torch.Size([32, 1, 28, 28])
label.shape --> torch.Size([32])


#### Build the Model

In [7]:
##
class FullyConnectedNetwork(nn.Module):
    def __init__(self, input_dim, hidden_dim, n_classes):
        super(FullyConnectedNetwork, self).__init__()
        ## create a network of two layers
        self.linear1 = nn.Linear(in_features=input_dim, out_features=hidden_dim)
        self.relu = nn.ReLU()   ## after the first layer
        self.linear2 = nn.Linear(in_features=hidden_dim, out_features=n_classes)
        ## no softmax at the end for multiclass --> cross entropy handle this
        
    def forward(self, x):
        out = self.linear1(x)
        out = self.relu(out)
        out = self.linear2(out)
        return out
    

## Training
print('Training started _________________________________________ \n')

model = FullyConnectedNetwork(input_size, hidden_size, num_classes).to(device=device)

## criteria
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

n_iterations = len(train_loader)     ## --> = len(train_dataset) / n_batch_size
n_train_samples = 0
n_train_correct = 0

for epoch in range(n_epochs):
    print(f'## epoch No. {epoch+1}')
    for i, (train_images, train_labels) in enumerate(train_loader):
        ## Here --> images shape are 100x1x28x28, but the Net needs --> batch_size*784
        ## we want to flatten as we use here FCN
        train_images = train_images.reshape(-1, 784).to(device=device)
        train_labels = train_labels.to(device=device)
        
        ## predicting then loss then backward then update weigths and empthy the gradeints
        y_pred_train = model(train_images)
        ## loss
        l = loss(y_pred_train, train_labels)
        ## backpropagation
        l.backward()
        ## optimization step
        optimizer.step()
        ## empty the gradients
        optimizer.zero_grad()

        ## during training
        ## get the max score --> return the value and index
        _, train_predictions = torch.max(y_pred_train, dim=1)
        
        n_train_samples += train_labels.shape[0]
        n_train_correct += (train_predictions==train_labels).sum().item()
        train_acc = (n_train_correct/n_train_samples)
        print(f'Iteration No. {i+1} | loss={l:.5f} | acc={train_acc:.5f}')
        
        
print('Training finished ___________________________________________ \n')        

Training started _________________________________________ 

## epoch No. 1
Iteration No. 1 | loss=2.32334 | acc=0.09375
Iteration No. 2 | loss=2.29553 | acc=0.09375
Iteration No. 3 | loss=2.28790 | acc=0.10417
Iteration No. 4 | loss=2.26094 | acc=0.13281
Iteration No. 5 | loss=2.23647 | acc=0.15000
Iteration No. 6 | loss=2.20145 | acc=0.16667
Iteration No. 7 | loss=2.20931 | acc=0.17857
Iteration No. 8 | loss=2.11318 | acc=0.20703
Iteration No. 9 | loss=2.09789 | acc=0.23611
Iteration No. 10 | loss=2.12051 | acc=0.25000
Iteration No. 11 | loss=2.09520 | acc=0.26420
Iteration No. 12 | loss=1.98189 | acc=0.29167
Iteration No. 13 | loss=2.00821 | acc=0.30529
Iteration No. 14 | loss=1.96870 | acc=0.32366
Iteration No. 15 | loss=1.93015 | acc=0.34583
Iteration No. 16 | loss=1.85767 | acc=0.36523
Iteration No. 17 | loss=1.91554 | acc=0.37500
Iteration No. 18 | loss=1.84655 | acc=0.38542
Iteration No. 19 | loss=1.80751 | acc=0.39803
Iteration No. 20 | loss=1.89425 | acc=0.40625
Iteration No.

Iteration No. 180 | loss=0.44411 | acc=0.78594
Iteration No. 181 | loss=0.37520 | acc=0.78643
Iteration No. 182 | loss=0.65949 | acc=0.78657
Iteration No. 183 | loss=0.57029 | acc=0.78689
Iteration No. 184 | loss=0.43140 | acc=0.78770
Iteration No. 185 | loss=0.33283 | acc=0.78834
Iteration No. 186 | loss=0.41007 | acc=0.78881
Iteration No. 187 | loss=0.23715 | acc=0.78977
Iteration No. 188 | loss=0.50948 | acc=0.79006
Iteration No. 189 | loss=0.39511 | acc=0.79051
Iteration No. 190 | loss=0.33914 | acc=0.79128
Iteration No. 191 | loss=0.53512 | acc=0.79156
Iteration No. 192 | loss=0.30385 | acc=0.79215
Iteration No. 193 | loss=0.43766 | acc=0.79258
Iteration No. 194 | loss=0.30777 | acc=0.79301
Iteration No. 195 | loss=0.38177 | acc=0.79391
Iteration No. 196 | loss=0.31279 | acc=0.79464
Iteration No. 197 | loss=0.36955 | acc=0.79553
Iteration No. 198 | loss=0.34004 | acc=0.79624
Iteration No. 199 | loss=0.54012 | acc=0.79648
Iteration No. 200 | loss=0.35372 | acc=0.79688
Iteration No.

Iteration No. 363 | loss=0.38703 | acc=0.84349
Iteration No. 364 | loss=0.49018 | acc=0.84358
Iteration No. 365 | loss=0.26495 | acc=0.84384
Iteration No. 366 | loss=0.30878 | acc=0.84392
Iteration No. 367 | loss=0.36149 | acc=0.84418
Iteration No. 368 | loss=0.66624 | acc=0.84409
Iteration No. 369 | loss=0.48373 | acc=0.84400
Iteration No. 370 | loss=0.34592 | acc=0.84409
Iteration No. 371 | loss=0.36112 | acc=0.84426
Iteration No. 372 | loss=0.23791 | acc=0.84442
Iteration No. 373 | loss=0.19140 | acc=0.84467
Iteration No. 374 | loss=0.24241 | acc=0.84492
Iteration No. 375 | loss=0.30489 | acc=0.84517
Iteration No. 376 | loss=0.26735 | acc=0.84525
Iteration No. 377 | loss=0.24871 | acc=0.84541
Iteration No. 378 | loss=0.24183 | acc=0.84557
Iteration No. 379 | loss=0.27111 | acc=0.84589
Iteration No. 380 | loss=0.24277 | acc=0.84613
Iteration No. 381 | loss=0.16261 | acc=0.84654
Iteration No. 382 | loss=0.34452 | acc=0.84670
Iteration No. 383 | loss=0.24205 | acc=0.84693
Iteration No.

Iteration No. 542 | loss=0.25810 | acc=0.86566
Iteration No. 543 | loss=0.35102 | acc=0.86573
Iteration No. 544 | loss=0.20913 | acc=0.86592
Iteration No. 545 | loss=0.31704 | acc=0.86600
Iteration No. 546 | loss=0.32305 | acc=0.86596
Iteration No. 547 | loss=0.16265 | acc=0.86609
Iteration No. 548 | loss=0.39906 | acc=0.86622
Iteration No. 549 | loss=0.39364 | acc=0.86629
Iteration No. 550 | loss=0.25405 | acc=0.86642
Iteration No. 551 | loss=0.49003 | acc=0.86632
Iteration No. 552 | loss=0.32741 | acc=0.86639
Iteration No. 553 | loss=0.38599 | acc=0.86635
Iteration No. 554 | loss=0.34224 | acc=0.86643
Iteration No. 555 | loss=0.38728 | acc=0.86644
Iteration No. 556 | loss=0.21051 | acc=0.86657
Iteration No. 557 | loss=0.34429 | acc=0.86653
Iteration No. 558 | loss=0.20653 | acc=0.86666
Iteration No. 559 | loss=0.58194 | acc=0.86661
Iteration No. 560 | loss=0.33214 | acc=0.86674
Iteration No. 561 | loss=0.55661 | acc=0.86670
Iteration No. 562 | loss=0.21645 | acc=0.86683
Iteration No.

Iteration No. 724 | loss=0.18310 | acc=0.87629
Iteration No. 725 | loss=0.12917 | acc=0.87642
Iteration No. 726 | loss=0.37170 | acc=0.87638
Iteration No. 727 | loss=0.28707 | acc=0.87642
Iteration No. 728 | loss=0.12539 | acc=0.87655
Iteration No. 729 | loss=0.12177 | acc=0.87671
Iteration No. 730 | loss=0.15910 | acc=0.87684
Iteration No. 731 | loss=0.18415 | acc=0.87692
Iteration No. 732 | loss=0.31337 | acc=0.87696
Iteration No. 733 | loss=0.22586 | acc=0.87705
Iteration No. 734 | loss=0.34509 | acc=0.87709
Iteration No. 735 | loss=0.14321 | acc=0.87717
Iteration No. 736 | loss=0.56033 | acc=0.87700
Iteration No. 737 | loss=0.25475 | acc=0.87708
Iteration No. 738 | loss=0.14996 | acc=0.87724
Iteration No. 739 | loss=0.28302 | acc=0.87724
Iteration No. 740 | loss=0.14011 | acc=0.87736
Iteration No. 741 | loss=0.26679 | acc=0.87740
Iteration No. 742 | loss=0.14364 | acc=0.87748
Iteration No. 743 | loss=0.17680 | acc=0.87757
Iteration No. 744 | loss=0.14928 | acc=0.87765
Iteration No.

...

Iteration No. 1114 | loss=0.00231 | acc=0.97467
Iteration No. 1115 | loss=0.00958 | acc=0.97467
Iteration No. 1116 | loss=0.00858 | acc=0.97467
Iteration No. 1117 | loss=0.00153 | acc=0.97467
Iteration No. 1118 | loss=0.00238 | acc=0.97467
Iteration No. 1119 | loss=0.01025 | acc=0.97468
Iteration No. 1120 | loss=0.00288 | acc=0.97468
Iteration No. 1121 | loss=0.00798 | acc=0.97468
Iteration No. 1122 | loss=0.00076 | acc=0.97468
Iteration No. 1123 | loss=0.10042 | acc=0.97468
Iteration No. 1124 | loss=0.00533 | acc=0.97468
Iteration No. 1125 | loss=0.02156 | acc=0.97468
Iteration No. 1126 | loss=0.03263 | acc=0.97468
Iteration No. 1127 | loss=0.00790 | acc=0.97469
Iteration No. 1128 | loss=0.00044 | acc=0.97469
Iteration No. 1129 | loss=0.00129 | acc=0.97469
Iteration No. 1130 | loss=0.01071 | acc=0.97469
Iteration No. 1131 | loss=0.00705 | acc=0.97469
Iteration No. 1132 | loss=0.07570 | acc=0.97469
Iteration No. 1133 | loss=0.00348 | acc=0.97469
Iteration No. 1134 | loss=0.01327 | acc=

Iteration No. 1292 | loss=0.01415 | acc=0.97486
Iteration No. 1293 | loss=0.02034 | acc=0.97486
Iteration No. 1294 | loss=0.00070 | acc=0.97486
Iteration No. 1295 | loss=0.01242 | acc=0.97486
Iteration No. 1296 | loss=0.00688 | acc=0.97486
Iteration No. 1297 | loss=0.00083 | acc=0.97487
Iteration No. 1298 | loss=0.02965 | acc=0.97487
Iteration No. 1299 | loss=0.00108 | acc=0.97487
Iteration No. 1300 | loss=0.09907 | acc=0.97487
Iteration No. 1301 | loss=0.00423 | acc=0.97487
Iteration No. 1302 | loss=0.02546 | acc=0.97487
Iteration No. 1303 | loss=0.00973 | acc=0.97487
Iteration No. 1304 | loss=0.00209 | acc=0.97487
Iteration No. 1305 | loss=0.00095 | acc=0.97487
Iteration No. 1306 | loss=0.01636 | acc=0.97488
Iteration No. 1307 | loss=0.00409 | acc=0.97488
Iteration No. 1308 | loss=0.02230 | acc=0.97488
Iteration No. 1309 | loss=0.02031 | acc=0.97488
Iteration No. 1310 | loss=0.01317 | acc=0.97488
Iteration No. 1311 | loss=0.05300 | acc=0.97488
Iteration No. 1312 | loss=0.03329 | acc=

Iteration No. 1473 | loss=0.00591 | acc=0.97506
Iteration No. 1474 | loss=0.06619 | acc=0.97506
Iteration No. 1475 | loss=0.02604 | acc=0.97506
Iteration No. 1476 | loss=0.01823 | acc=0.97506
Iteration No. 1477 | loss=0.00700 | acc=0.97506
Iteration No. 1478 | loss=0.00387 | acc=0.97506
Iteration No. 1479 | loss=0.03656 | acc=0.97506
Iteration No. 1480 | loss=0.02375 | acc=0.97506
Iteration No. 1481 | loss=0.04188 | acc=0.97506
Iteration No. 1482 | loss=0.00154 | acc=0.97507
Iteration No. 1483 | loss=0.00217 | acc=0.97507
Iteration No. 1484 | loss=0.06774 | acc=0.97507
Iteration No. 1485 | loss=0.01519 | acc=0.97507
Iteration No. 1486 | loss=0.00151 | acc=0.97507
Iteration No. 1487 | loss=0.06230 | acc=0.97507
Iteration No. 1488 | loss=0.00943 | acc=0.97507
Iteration No. 1489 | loss=0.04148 | acc=0.97507
Iteration No. 1490 | loss=0.02516 | acc=0.97507
Iteration No. 1491 | loss=0.00299 | acc=0.97507
Iteration No. 1492 | loss=0.00305 | acc=0.97508
Iteration No. 1493 | loss=0.00284 | acc=

Iteration No. 1645 | loss=0.00543 | acc=0.97520
Iteration No. 1646 | loss=0.01260 | acc=0.97520
Iteration No. 1647 | loss=0.00299 | acc=0.97521
Iteration No. 1648 | loss=0.00115 | acc=0.97521
Iteration No. 1649 | loss=0.02154 | acc=0.97521
Iteration No. 1650 | loss=0.00014 | acc=0.97521
Iteration No. 1651 | loss=0.00818 | acc=0.97521
Iteration No. 1652 | loss=0.02490 | acc=0.97521
Iteration No. 1653 | loss=0.01057 | acc=0.97521
Iteration No. 1654 | loss=0.01532 | acc=0.97521
Iteration No. 1655 | loss=0.04268 | acc=0.97521
Iteration No. 1656 | loss=0.00869 | acc=0.97522
Iteration No. 1657 | loss=0.13732 | acc=0.97521
Iteration No. 1658 | loss=0.02010 | acc=0.97521
Iteration No. 1659 | loss=0.04415 | acc=0.97521
Iteration No. 1660 | loss=0.01668 | acc=0.97522
Iteration No. 1661 | loss=0.01498 | acc=0.97522
Iteration No. 1662 | loss=0.08726 | acc=0.97522
Iteration No. 1663 | loss=0.06957 | acc=0.97522
Iteration No. 1664 | loss=0.01091 | acc=0.97522
Iteration No. 1665 | loss=0.00201 | acc=

Iteration No. 1824 | loss=0.01068 | acc=0.97535
Iteration No. 1825 | loss=0.00066 | acc=0.97535
Iteration No. 1826 | loss=0.00544 | acc=0.97535
Iteration No. 1827 | loss=0.00545 | acc=0.97535
Iteration No. 1828 | loss=0.00502 | acc=0.97535
Iteration No. 1829 | loss=0.00158 | acc=0.97535
Iteration No. 1830 | loss=0.02469 | acc=0.97535
Iteration No. 1831 | loss=0.02345 | acc=0.97536
Iteration No. 1832 | loss=0.01732 | acc=0.97536
Iteration No. 1833 | loss=0.01108 | acc=0.97536
Iteration No. 1834 | loss=0.10840 | acc=0.97536
Iteration No. 1835 | loss=0.05153 | acc=0.97536
Iteration No. 1836 | loss=0.01129 | acc=0.97536
Iteration No. 1837 | loss=0.00789 | acc=0.97536
Iteration No. 1838 | loss=0.01276 | acc=0.97536
Iteration No. 1839 | loss=0.00126 | acc=0.97536
Iteration No. 1840 | loss=0.03071 | acc=0.97536
Iteration No. 1841 | loss=0.00266 | acc=0.97536
Iteration No. 1842 | loss=0.00515 | acc=0.97536
Iteration No. 1843 | loss=0.00653 | acc=0.97537
Iteration No. 1844 | loss=0.00179 | acc=

#### Evaluation

In [8]:
## when you evaluate you must stop gradients to not kick the requires_grad=True
n_test_correct = 0
n_test_samples = 0

with torch.no_grad():
    for i, (test_images, test_labels) in enumerate(test_loader):
        test_images = test_images.reshape(-1, 784).to(device=device)
        test_labels = test_labels.to(device=device)

        y_pred_test = model(test_images)

        ## get the max score --> return the value and index
        _, test_predictions = torch.max(y_pred_test, dim=1)
        
        n_test_samples += test_labels.shape[0]
        n_test_correct += (test_predictions==test_labels).sum().item()
        
        
test_acc = (n_test_correct/n_test_samples) * 100.0
print(f'Test Accuracy is .. {test_acc} %')

Test Accuracy is .. 97.68 %


#### Done!